In [21]:
from scipy import sparse
import numpy as np

def make_derivative_matrix(w, h):
    """Creates a matrixform of the first order differencing.

    Args:
        w: width of the image to be differentiated
        h: height of the image to be differentiated

    Returns:
        First order differencing in matrix form.
    """
    def generate_D_matrix(n):
        e = np.ones([2,n])
        e[1,:] = -e[1,:] 
        return sparse.spdiags(e, [0,1], n, n)

    Dy = sparse.kron(sparse.eye(w), generate_D_matrix(h)) 
    Dx = sparse.kron(generate_D_matrix(w),sparse.eye(h)) 

    D = sparse.vstack([Dy, Dx])
    return D


def derivative(array, deriv_matrix):
    """Creates the derivative of an input array.

    Args: 
        array: input with dimension: [class, height, width]

    Returns:
        Derivative [2, class, height, width] of the input array.
    """
    c, h, w = array.shape
    dxy = deriv_matrix @ np.transpose(array, (2, 1, 0)).reshape(-1,c)
    dxy = np.transpose(dxy.reshape(2, w, h, c), (0, 3, 2, 1))
    return dxy

In [24]:
deriv = make_derivative_matrix(3, 2)
I = np.array([[1, 4, 2], [5, 3, 2]]).reshape((1, 2, 3))
derivative(I, deriv)

array([[[[-4.,  1.,  0.],
         [ 5.,  3.,  2.]]],


       [[[-3.,  2.,  2.],
         [ 2.,  1.,  2.]]]])

In [25]:
from scipy.ndimage import gaussian_filter, laplace

image_first_derivative = gaussian_filter(I, sigma=1)

In [57]:
I = np.array([[[1, 4, 2], [5, 3, 2]], [[4, 1, 6], [2, 1, 1]]])
I2 = np.array([[1, 4, 2], [5, 3, 2]])

In [70]:
I

array([[[1, 4, 2],
        [5, 3, 2]],

       [[4, 1, 6],
        [2, 1, 1]]])

In [63]:
I[:,-1,:]

array([[5, 3, 2],
       [2, 1, 1]])

In [65]:
np.diff(I, axis = 1)

array([[[ 4, -1,  0]],

       [[-2,  0, -5]]])

In [59]:
#np.concatenate((np.diff(I2, axis = 0),np.array([[0, 0, 0]])), axis = 0)
np.concatenate((np.diff(I2, axis = 1),np.array([[0],[0]])), axis = 1)

array([[ 3, -2,  0],
       [-2, -1,  0]])

In [68]:
np.concatenate((np.diff(I, axis = 1), np.expand_dims(I[:,-1,:], 1)), axis = 1)

array([[[ 4, -1,  0],
        [ 5,  3,  2]],

       [[-2,  0, -5],
        [ 2,  1,  1]]])

In [74]:
def compute_deriv(img):
    diffs_x = np.diff(img, axis = 2)
    diffs_y = np.diff(img, axis = 1)
    
    last_cols_reshaped = np.expand_dims(img[:,:,-1], 2)
    last_rows_reshaped = np.expand_dims(img[:,-1,:], 1)

    dx = np.concatenate((diffs_x, last_cols_reshaped), axis = 2)
    dy = np.concatenate((diffs_y, last_rows_reshaped), axis = 1)

    return np.array([dx, dy])

(2, 2, 2, 3)

# Testing our functions

In [1]:
from priorEstimator import PriorEstimator
import numpy as np

In [2]:
I = np.array([[[1, 4, 2], [5, 3, 2]], [[4, 1, 6], [2, 1, 1]], [[1, 4, 1], [4, 3, 7]]])
Theta = np.array([[[0, 1, 0], [0, 1, 1]], [[0, 0, 1], [1, 0, 0]], [[1, 0, 0], [0, 0, 0]]])

In [3]:
estim = PriorEstimator(I)

In [8]:
estim._prior_energy(I, gamma = 5, theta= Theta)

1.1971310567689821